In [1]:
# 역전파 자동화

In [55]:
class Variable:
    def __init__(self, data):
        self.data = data
        self.grad = None
        self.creator = None

    def set_creator(self, func):
        self.creator = func

    def backward(self):
        f = self.creator # 이부분이 None이 될 때까지
        if f is not None:
            x = f.input
            x.grad = f.backward(self.grad)
            x.backward() # 다시 backward 메소드 호출한다.

In [56]:
class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        output = Variable(y)
        output.set_creator(self) # 출력 변수에 창조자를 설정한다.
        print(output.set_creator(self))
        self.input = input
        self.ouput = output # 출력도 저장한다.
        return output

    def forward(self, x):
        raise NotImplementedError()

    def backward(self, gy):
        raise  NotImplementedError()

In [57]:
import numpy as np

class Exp(Function):
    def forward(self, x):
        y = np.exp(x)
        return y

    def backward(self, gy):
        x = self.input.data
        print(x)
        gx = np.exp(x) * gy
        return gx

In [58]:
class Square(Function):
    def forward(self, x):
        y = x ** 2
        return y

    def backward(self, gy):
        x = self.input.data
        print(x)
        gx = 2 * x * gy #
        return gx

In [59]:
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))

In [60]:
a = A(x)
b = B(a)
y = C(b)


None
None
None


In [61]:
assert y.creator == C
assert y.creator.input == b
assert y.creator.input.creator == B
assert y.creator.input.creator.input == a
assert y.creator.input.creator.input.creator == A
assert y.creator.input.creator.input.creator.input == x

In [62]:
y.grad = np.array(1.0)

C = y.creator # 함수를 가져온다
b = C.input # 함수의 입력을 가져온다.
b.grad = C.backward(y.grad) # 함수의 backward 메서드를 호출한다.
print(b.grad)

1.2840254166877414
2.568050833375483


In [63]:
B = b.creator
a = B.input
a.grad = B.backward(b.grad)
print(a.grad)

0.25
3.297442541400256


In [64]:
A = a.creator
x = A.input
x.grad = A.backward(a.grad)
print(x.grad)

0.5
3.297442541400256


In [65]:
y.grad = np.array(1.0)
y.backward()
print(x.grad)

1.2840254166877414
0.25
0.5
3.297442541400256
